In [1]:
#r "nuget: StackExchange.Redis"


Installed Packages StackExchange.Redis, 2.7.10

In [18]:
using StackExchange.Redis;
using System;
using System.Threading;

var options = ConfigurationOptions.Parse("78.109.201.86:6379"); // host1:port1, host2:port2, ...
options.Password = "1qazxsw2$$";

string redisConnectionString = "localhost:6379";
string channelName = "sampleChannel"; // simulation ID.ch
// string channelName = "3e37381b-32bd-4684-8953-08dc093dd85d.ch"; // simulation ID.ch
string streamKey = "sampleStream"; 

ConnectionMultiplexer redis = await ConnectionMultiplexer.ConnectAsync(options);
// ConnectionMultiplexer redis = await ConnectionMultiplexer.ConnectAsync(redisConnectionString);
ISubscriber subscriber = redis.GetSubscriber();
await subscriber.PublishAsync(channelName, "Keylie_connect"); // call fuction 
await subscriber.PublishAsync(channelName, "Hello from Server 2!");
await subscriber.PublishAsync(channelName, "Hello from Server 3!");
await subscriber.PublishAsync(channelName, "Hello from Server 4!");

redis.Dispose();


In [3]:
using StackExchange.Redis;
using System;

var options = ConfigurationOptions.Parse("78.109.201.86:6379"); // host1:port1, host2:port2, ...
options.Password = "1qazxsw2$$";

string redisConnectionString = "localhost:6379";
string channelName = "sampleChannel";
string streamKey = "sampleStream";

ConnectionMultiplexer redis = await ConnectionMultiplexer.ConnectAsync(options);
// ConnectionMultiplexer redis = await ConnectionMultiplexer.ConnectAsync(redisConnectionString);
ISubscriber subscriber = redis.GetSubscriber();

subscriber.Subscribe(channelName, (channel, message) =>
{
    Console.WriteLine($"Received message from channel '{channel}': {message}");
});

await subscriber.PublishAsync(channelName, "Hello from Server 1!");

redis.Dispose();


Received message from channel 'sampleChannel': Hello from Server 1!


In [14]:
using StackExchange.Redis;
using System;

// var options = ConfigurationOptions.Parse("78.109.201.86:6379"); // host1:port1, host2:port2, ...
// options.Password = "1qazxsw2$$";

string redisConnectionString = "localhost:6379";
string channelName = "sampleChannel";
string streamKey = "sampleStream";
string consumerGroupName = "sampleGroup";

ConnectionMultiplexer redis = await ConnectionMultiplexer.ConnectAsync(redisConnectionString);

IDatabase redisDb = redis.GetDatabase();

var message = new NameValueEntry("messageField", "Message from Server 1");

var messageId = await redisDb.StreamAddAsync(streamKey, new [] {message});
Console.WriteLine($"Added message to stream '{streamKey}' with ID: {messageId}");

// Create a consumer group associated with the stream
try{
    redisDb.StreamCreateConsumerGroup(streamKey, consumerGroupName, StreamPosition.NewMessages);
}catch (Exception e){
    Console.WriteLine($"consumer group already exist 'I THINK' : {e.Message}");
}

// this is not accurate (always returning 4)
RedisResult result = await redisDb.ExecuteAsync("XPENDING", streamKey, consumerGroupName);
Console.WriteLine($"Total pending messages: {result}");

var streamMessages = await redisDb.StreamReadAsync(streamKey , "0");
foreach (var message in streamMessages)
{
    Console.WriteLine($"Received message from stream '{streamKey}': ID:{message.Id} VAL0:{message.Values[0]}");
    var t = await redisDb.StreamAcknowledgeAsync(streamKey , consumerGroupName, message.Id);
}
// Delete acknowledged messages from the stream
await redisDb.StreamDeleteAsync(streamKey, streamMessages.Select(msg => msg.Id).ToArray());

redis.Dispose();


Added message to stream 'sampleStream' with ID: 1704110598072-0
consumer group already exist 'I THINK' : BUSYGROUP Consumer Group name already exists
Total pending messages: 4 element(s)
Received message from stream 'sampleStream': ID:1704110587263-0 VAL0:messageField: Message from Server 1
Received message from stream 'sampleStream': ID:1704110588289-0 VAL0:messageField: Message from Server 1
Received message from stream 'sampleStream': ID:1704110589577-0 VAL0:messageField: Message from Server 1
Received message from stream 'sampleStream': ID:1704110596155-0 VAL0:messageField: Message from Server 1
Received message from stream 'sampleStream': ID:1704110597178-0 VAL0:messageField: Message from Server 1
Received message from stream 'sampleStream': ID:1704110598072-0 VAL0:messageField: Message from Server 1
